# Linear Programming using PULP demo

In this notebook, we will show how we can use the pulp package in python to formulate the linear programming optimization problem. In diet problem, we want to minimize the cost of food while fulfilling the requirement of nnutrients on daily diet. We will use the dummy data of foods for this demo.  

**Model class** - Optimization

**Optimization type** - Linear program w integer contraints


**Purpose** - LP problem formulation using pulp

**Problem** - Suggesting diet with minimum cost

## Dataset 
We are given following information in the dataset - 
- 1. **Nutrient (Carbs, Proteins, Vit A, Vit C, Iron, Calcium etc) content in each serving of given food**
- 2. **Price per servingg of the food** 
- 3. **Daily requirements ranges for each nutrient**

The dataset is a very small 64 rows dataset, taken from Morgridge Institute for Research, Inc. (“MIR”), a private not-for-profit research organization. This dataset should only be used for demo, training, teaching, education purposes and should not be used for commercial purposes. Link - https://neos-guide.org/content/diet-problem-solver



## Basics of LP
In any linear program, we have three major components. We will talk about them with respecct to foods dataset - 
### 1. Variables 
We have to formulate the linear programing problem and for that we need to first define the variables in this problem. Here our variables will be **quantity of food** served. We want to know what ingredient and in what quanntity should be selected to fulfill the daily requirements. We know that these quantity of per serving can be fraction and can be integer as well. So we will first solve it for fractional quantities. We will give two solutions 


### 2. Objective function
We want to minimize the cost in this problem so our objective function will be the cost values. We calculate the cost as - 
- Cost = sum(price_per_serving for food * Quantity consumed (i.e. servings used))

### 3. Constraints 
We know that the values of nnutrients has lower and upper bounds. So it will depend of the servings of food one is eating. If a person is eating food_A, Food_B in 2,3 quantities erespectively then the nutruients he/she will be consuming is 
- Nutrient = 2* nutrient_in_food_A + 3*Nutrients_in_food_B
- min < Nutrient < max



## Dependancies  
- Pulp
- numpy 
- pandas 


## Suggesting diet with minimum cost

In [1]:
import pandas as pd 
import numpy as np 
from pulp import * 

In [2]:
d_df = pd.read_csv("diet.csv")
d_df.shape

(67, 14)

In [3]:
d_df.head(20)

Foods Price/ Serving          Serving Size  Calories  \
0       Frozen Broccoli         $0.16              10 Oz Pkg      73.8   
1           Carrots,Raw         $0.07       1/2 Cup Shredded      23.7   
2           Celery, Raw         $0.04                1 Stalk       6.4   
3           Frozen Corn         $0.18                1/2 Cup      72.2   
4   Lettuce,Iceberg,Raw         $0.02                 1 Leaf       2.6   
5   Peppers, Sweet, Raw         $0.53               1 Pepper      20.0   
6       Potatoes, Baked         $0.06                1/2 Cup     171.5   
7                  Tofu         $0.31              1/4 block      88.2   
8       Roasted Chicken         $0.84           1 lb chicken     277.4   
9    Spaghetti W/ Sauce         $0.78              1 1/2 Cup     358.2   
10  Tomato,Red,Ripe,Raw         $0.27     1 Tomato, 2-3/5 In      25.8   
11     Apple,Raw,W/Skin         $0.24     1 Fruit,3/Lb,Wo/Rf      81.4   
12               Banana         $0.15   1 Fruit,Wo/Skn&Seeds     104.9   
13               Grapes         $0.32        10 Fruits,Wo/Rf      15.1   
14  Kiwifruit,Raw,Fresh         $0.49      1 Med Frt,Wo/Skin      46.4   
15              Oranges         $0.15       1 Frt,2-5/8 Diam      61.6   
16               Bagels         $0.16                   1 Oz      78.0   
17          Wheat Bread         $0.05                   1 Sl      65.0   
18          White Bread         $0.06                   1 Sl      65.0   
19      Oatmeal Cookies         $0.09               1 Cookie      81.0   

    Cholesterol mg  Total_Fat g  Sodium mg  Carbohydrates g  Dietary_Fiber g  \
0              0.0          0.8       68.2             13.6              8.5   
1              0.0          0.1       19.2              5.6              1.6   
2              0.0          0.1       34.8              1.5              0.7   
3              0.0          0.6        2.5             17.1              2.0   
4              0.0          0.0        1.8              0.4              0.3   
5              0.0          0.1        1.5              4.8              1.3   
6              0.0          0.2       15.2             39.9              3.2   
7              0.0          5.5        8.1              2.2              1.4   
8            129.9         10.8      125.6              0.0              0.0   
9              0.0         12.3     1237.1             58.3             11.6   
10             0.0          0.4       11.1              5.7              1.4   
11             0.0          0.5        0.0             21.0              3.7   
12             0.0          0.5        1.1             26.7              2.7   
13             0.0          0.1        0.5              4.1              0.2   
14             0.0          0.3        3.8             11.3              2.6   
15             0.0          0.2        0.0             15.4              3.1   
16             0.0          0.5      151.4             15.1              0.6   
17             0.0          1.0      134.5             12.4              1.3   
18             0.0          1.0      132.5             11.8              1.1   
19             0.0          3.3       68.9             12.4              0.6   

    Protein g  Vit_A IU  Vit_C IU  Calcium mg  Iron mg  
0         8.0    5867.4     160.2       159.0      2.3  
1         0.6   15471.0       5.1        14.9      0.3  
2         0.3      53.6       2.8        16.0      0.2  
3         2.5     106.6       5.2         3.3      0.3  
4         0.2      66.0       0.8         3.8      0.1  
5         0.7     467.7      66.1         6.7      0.3  
6         3.7       0.0      15.6        22.7      4.3  
7         9.4      98.6       0.1       121.8      6.2  
8        42.2      77.4       0.0        21.9      1.8  
9         8.2    3055.2      27.9        80.2      2.3  
10        1.0     766.3      23.5         6.2      0.6  
11        0.3      73.1       7.9         9.7      0.2  
12        1.2      92.3      10.4        

### Daily requirements of nutrients
We are given daily requirement values of nutrients in last few row. We check these values and create a dictionary that has minimum and maximum value of a particular nutrient. Following cell create this dictionary. We call it req (short of requirements)

Below cell contains has nutritional constraints values for single person. Here whenever one eats a food, one basically gain nutrients like calcium, iron, protein etc and the cell below represents minimum and maximum constraints for these nutients. 

In [4]:

req = {}
req['Calories'] = {'min':1500, 'max':2500}
req['Sodium_mg'] = {'min':800, 'max':2000}
req['Carbohydrates_g'] = {'min':130, 'max':450}
req['Vit_A_IU'] = {'min':1000, 'max':10000}
req['Vit_C_IU'] = {'min':400, 'max':5000}
req['Calcium_mg'] = {'min':700, 'max':1500}
req['Iron_mg'] = {'min':10, 'max':40}
req['Cholesterol_mg'] = {'min':30, 'max':240}
req['Total_Fat_g'] = {'min':20, 'max':70}
req['Dietary_Fiber_g'] = {'min':125, 'max':250}
req['Protein_g'] = {'min':60, 'max':100}
req

{'Calories': {'min': 1500, 'max': 2500},
 'Sodium_mg': {'min': 800, 'max': 2000},
 'Carbohydrates_g': {'min': 130, 'max': 450},
 'Vit_A_IU': {'min': 1000, 'max': 10000},
 'Vit_C_IU': {'min': 400, 'max': 5000},
 'Calcium_mg': {'min': 700, 'max': 1500},
 'Iron_mg': {'min': 10, 'max': 40},
 'Cholesterol_mg': {'min': 30, 'max': 240},
 'Total_Fat_g': {'min': 20, 'max': 70},
 'Dietary_Fiber_g': {'min': 125, 'max': 250},
 'Protein_g': {'min': 60, 'max': 100}}

We will now create lists of all the given nutrients and then we will use them in contraints 

In [5]:
# creating lists of all the given columns in data 
# here we will vectorize the given information from dataframe in order to feed it to pulp optimizer. 

d_df.dropna(inplace = True)
foods = list(d_df.Foods)
foods

costs_dollar = list(d_df['Price/ Serving'].values)
costs = [float(x[1:]) for x in costs_dollar]
costs

#calories = list(d_df.Calories.values)
Calcium_mg = list(d_df['Calcium mg'].values)
Calcium_mg

Carbohydrates_g = list(d_df['Carbohydrates g'].values)
Carbohydrates_g

Iron_mg = list(d_df['Iron mg'].values)
Iron_mg

Sodium_mg = list(d_df['Sodium mg'].values)
Sodium_mg

Vit_A_IU = list(d_df['Vit_A IU'].values)
Vit_A_IU


Vit_C_IU = list(d_df['Vit_C IU'].values)
Vit_C_IU

Calories = list(d_df['Calories'].values)
Calories

Protein_g = list(d_df['Protein g'].values)
Dietary_Fiber_g = list(d_df['Dietary_Fiber g'].values)
Total_Fat_g = list(d_df['Total_Fat g'].values)
Cholesterol_mg = list(d_df['Cholesterol mg'].values)



### 1. Variables 
Lets create thevariables for linear program. In pulp, we can create variables using LpVariables and create dictionaries of variables.  


In [6]:
# Here we are adding food prefixx to add the food items and create foods Lpvariable
food_vars = LpVariable.dicts("Food",foods,0,None, "Integer") # integer as we are solving for integer quantities
# Here we are creating LP variables whose names starts with "Food" for each food whose values 
# go from 0 to infinity and integer values are allowed
prob = LpProblem("Diet_Optimization", LpMinimize) # LPminimize as we want to minimize the cost

### 2. Objective function
 
- Cost = sum(price_per_serving for food * Quantity consumed (i.e. servings used))

In [7]:
prob += lpSum([costs[foods.index(i)]*food_vars[i] for i in foods]) # here we create Objective function for this problem 

### 3. Constraints 



- min_Calcium_permisible <= $\sum_{from \mspace{2mu} all \mspace{2mu} foods \mspace{2mu} in \mspace{2mu} diet}^{}$Calcium <= max_Calcium_permisible 



- min_Carbohydrates_permisible <= $\sum_{from \mspace{2mu} all \mspace{2mu} foods \mspace{2mu} in \mspace{2mu} diet}^{}$Carbohydrates <= max_Carbohydrates_permisible 



- min_Iron_permisible <= $\sum_{from \mspace{2mu} all \mspace{2mu} foods \mspace{2mu} in \mspace{2mu} diet}^{}$Iron <= max_Iron_permisible 



- min_Sodium_permisible <= $\sum_{from \mspace{2mu} all \mspace{2mu} foods \mspace{2mu} in \mspace{2mu} diet}^{}$Sodium <= max_Sodium_permisible 



- min_Vit_A_permisible <= $\sum_{from \mspace{2mu} all \mspace{2mu} foods \mspace{2mu} in \mspace{2mu} diet}^{}$Vit_A <= max_Vit_A_permisible 



- min_Vit_C_permisible <= $\sum_{from \mspace{2mu} all \mspace{2mu} foods \mspace{2mu} in \mspace{2mu} diet}^{}$Vit_C <= max_Vit_C_permisible 



- min_Calories_permisible <= $\sum_{from \mspace{2mu} all \mspace{2mu} foods \mspace{2mu} in \mspace{2mu} diet}^{}$Calories <= max_Calories_permisible 



- min_Protein_permisible <= $\sum_{from \mspace{2mu} all \mspace{2mu} foods \mspace{2mu} in \mspace{2mu} diet}^{}$Protein <= max_Protein_permisible 



- min_Dietary_Fiber_permisible <= $\sum_{from \mspace{2mu} all \mspace{2mu} foods \mspace{2mu} in \mspace{2mu} diet}^{}$Dietary_Fiber <= max_Dietary_Fiber_permisible 



- min_Total_Fat_permisible <= $\sum_{from \mspace{2mu} all \mspace{2mu} foods \mspace{2mu} in \mspace{2mu} diet}^{}$Total_Fat <= max_Total_Fat_permisible 



- min_Cholesterol_permisible <= $\sum_{from \mspace{2mu} all \mspace{2mu} foods \mspace{2mu} in \mspace{2mu} diet}^{}$Cholesterol <= max_Cholesterol_permisible 





 

In [8]:
# Here we will add constraints to the LP problem 
# constraints are related to nutrients - i.e. for the given diet, the value of nutrient should be between 
# minimum and maximum value permisible 

prob += lpSum([Calcium_mg[foods.index(i)] * food_vars[i] for i in foods]) >= req["Calcium_mg"]['min']
prob += lpSum([Calcium_mg[foods.index(i)] * food_vars[i] for i in foods]) <= req["Calcium_mg"]['max']


prob += lpSum([Carbohydrates_g[foods.index(i)] * food_vars[i] for i in foods]) >= req["Carbohydrates_g"]['min']
prob += lpSum([Carbohydrates_g[foods.index(i)] * food_vars[i] for i in foods]) <= req["Carbohydrates_g"]['max']

prob += lpSum([Iron_mg[foods.index(i)] * food_vars[i] for i in foods]) >= req["Iron_mg"]['min']
prob += lpSum([Iron_mg[foods.index(i)] * food_vars[i] for i in foods]) <= req["Iron_mg"]['max']

prob += lpSum([Sodium_mg[foods.index(i)] * food_vars[i] for i in foods]) >= req["Sodium_mg"]['min']
prob += lpSum([Sodium_mg[foods.index(i)] * food_vars[i] for i in foods]) <= req["Sodium_mg"]['max']

prob += lpSum([Vit_A_IU[foods.index(i)] * food_vars[i] for i in foods]) >= req["Vit_A_IU"]['min']
prob += lpSum([Vit_A_IU[foods.index(i)] * food_vars[i] for i in foods]) <= req["Vit_A_IU"]['max']

prob += lpSum([Vit_C_IU[foods.index(i)] * food_vars[i] for i in foods]) >= req["Vit_C_IU"]['min']
prob += lpSum([Vit_C_IU[foods.index(i)] * food_vars[i] for i in foods]) <= req["Vit_C_IU"]['max']

prob += lpSum([Calories[foods.index(i)] * food_vars[i] for i in foods]) >= req["Calories"]['min']
prob += lpSum([Calories[foods.index(i)] * food_vars[i] for i in foods]) <= req["Calories"]['max']

prob += lpSum([Protein_g [foods.index(i)] * food_vars[i] for i in foods]) >= req["Protein_g"]['min']
prob += lpSum([Protein_g [foods.index(i)] * food_vars[i] for i in foods]) <= req["Protein_g"]['max']

prob += lpSum([Dietary_Fiber_g[foods.index(i)] * food_vars[i] for i in foods]) >= req["Dietary_Fiber_g"]['min']
prob += lpSum([Dietary_Fiber_g[foods.index(i)] * food_vars[i] for i in foods]) <= req["Dietary_Fiber_g"]['max']

prob += lpSum([Total_Fat_g[foods.index(i)] * food_vars[i] for i in foods]) >= req["Total_Fat_g"]['min']
prob += lpSum([Total_Fat_g[foods.index(i)] * food_vars[i] for i in foods]) <= req["Total_Fat_g"]['max']

prob += lpSum([Cholesterol_mg[foods.index(i)] * food_vars[i] for i in foods]) >= req["Cholesterol_mg"]['min']
prob += lpSum([Cholesterol_mg[foods.index(i)] * food_vars[i] for i in foods]) <= req["Cholesterol_mg"]['max']


In [9]:
prob.solve() # Here he solve the solver, It shows 1 - that means optimum solution is found

1

In [10]:
print("Status:", LpStatus[prob.status]) # It shows status as optimum

Status: Optimal


In [11]:
for v in prob.variables():
    if v.varValue> 0: # We print only quantities/servings which are non-zero
        print(v.name, "=", v.varValue)

Food_Celery,_Raw = 41.0
Food_Kiwifruit,Raw,Fresh = 1.0
Food_Lettuce,Iceberg,Raw = 91.0
Food_Oranges = 2.0
Food_Poached_Eggs = 1.0
Food_Popcorn,Air_Popped = 14.0


**Conclusion** - we can see that all the foods are in integer quantities and food that we got in solution satisfy nutrition requirement with lowest cost.

In [12]:
print("Total Cost of Ingredients per can = ", value(prob.objective))


Total Cost of Ingredients per can =  4.890000000000001


## This Solution with fractional servings

In [13]:
# Solution for continous servings - 
food_vars = LpVariable.dicts("Food",foods,0,None, "Continuous")
prob = LpProblem("Diet_Optimization", LpMinimize)
prob += lpSum([costs[foods.index(i)]*food_vars[i] for i in foods]) # here we create Objecctive function for this problem 

prob += lpSum([Calcium_mg[foods.index(i)] * food_vars[i] for i in foods]) >= req["Calcium_mg"]['min']
prob += lpSum([Calcium_mg[foods.index(i)] * food_vars[i] for i in foods]) <= req["Calcium_mg"]['max']


prob += lpSum([Carbohydrates_g[foods.index(i)] * food_vars[i] for i in foods]) >= req["Carbohydrates_g"]['min']
prob += lpSum([Carbohydrates_g[foods.index(i)] * food_vars[i] for i in foods]) <= req["Carbohydrates_g"]['max']

prob += lpSum([Iron_mg[foods.index(i)] * food_vars[i] for i in foods]) >= req["Iron_mg"]['min']
prob += lpSum([Iron_mg[foods.index(i)] * food_vars[i] for i in foods]) <= req["Iron_mg"]['max']

prob += lpSum([Sodium_mg[foods.index(i)] * food_vars[i] for i in foods]) >= req["Sodium_mg"]['min']
prob += lpSum([Sodium_mg[foods.index(i)] * food_vars[i] for i in foods]) <= req["Sodium_mg"]['max']

prob += lpSum([Vit_A_IU[foods.index(i)] * food_vars[i] for i in foods]) >= req["Vit_A_IU"]['min']
prob += lpSum([Vit_A_IU[foods.index(i)] * food_vars[i] for i in foods]) <= req["Vit_A_IU"]['max']

prob += lpSum([Vit_C_IU[foods.index(i)] * food_vars[i] for i in foods]) >= req["Vit_C_IU"]['min']
prob += lpSum([Vit_C_IU[foods.index(i)] * food_vars[i] for i in foods]) <= req["Vit_C_IU"]['max']

prob += lpSum([Calories[foods.index(i)] * food_vars[i] for i in foods]) >= req["Calories"]['min']
prob += lpSum([Calories[foods.index(i)] * food_vars[i] for i in foods]) <= req["Calories"]['max']

prob += lpSum([Protein_g [foods.index(i)] * food_vars[i] for i in foods]) >= req["Protein_g"]['min']
prob += lpSum([Protein_g [foods.index(i)] * food_vars[i] for i in foods]) <= req["Protein_g"]['max']

prob += lpSum([Dietary_Fiber_g[foods.index(i)] * food_vars[i] for i in foods]) >= req["Dietary_Fiber_g"]['min']
prob += lpSum([Dietary_Fiber_g[foods.index(i)] * food_vars[i] for i in foods]) <= req["Dietary_Fiber_g"]['max']

prob += lpSum([Total_Fat_g[foods.index(i)] * food_vars[i] for i in foods]) >= req["Total_Fat_g"]['min']
prob += lpSum([Total_Fat_g[foods.index(i)] * food_vars[i] for i in foods]) <= req["Total_Fat_g"]['max']

prob += lpSum([Cholesterol_mg[foods.index(i)] * food_vars[i] for i in foods]) >= req["Cholesterol_mg"]['min']
prob += lpSum([Cholesterol_mg[foods.index(i)] * food_vars[i] for i in foods]) <= req["Cholesterol_mg"]['max']



prob.solve() 
for v in prob.variables():
    if v.varValue> 0: # We print only quantities/servings which are non-zero
        print(v.name, "=", v.varValue)

Food_Celery,_Raw = 52.64371
Food_Frozen_Broccoli = 0.25960653
Food_Lettuce,Iceberg,Raw = 63.988506
Food_Oranges = 2.2929389
Food_Poached_Eggs = 0.14184397
Food_Popcorn,Air_Popped = 13.869322


**Conclusion** - Here we get foods in fractional quantities

In [14]:
print("Total Cost of Ingredients per can = ", value(prob.objective))

Total Cost of Ingredients per can =  4.337116797399999


## Part 2
#### (A) Here along with the given contraints, we introduce a few extra contraints like (a) **If a food is selected at least 0.1 fraction has to be selected.**

In [23]:
# Part2  (a)
food_vars = LpVariable.dicts("Food",foods,0,None, "Continuous")
prob1 = LpProblem("Diet_Optimization", LpMinimize)

prob1 += lpSum([costs[foods.index(i)]*food_vars[i] for i in foods])

prob1 += lpSum([Calcium_mg[foods.index(i)] * food_vars[i] for i in foods]) >= req["Calcium_mg"]['min']
prob1 += lpSum([Calcium_mg[foods.index(i)] * food_vars[i] for i in foods]) <= req["Calcium_mg"]['max']


prob1 += lpSum([Carbohydrates_g[foods.index(i)] * food_vars[i] for i in foods]) >= req["Carbohydrates_g"]['min']
prob1 += lpSum([Carbohydrates_g[foods.index(i)] * food_vars[i] for i in foods]) <= req["Carbohydrates_g"]['max']

prob1 += lpSum([Iron_mg[foods.index(i)] * food_vars[i] for i in foods]) >= req["Iron_mg"]['min']
prob1 += lpSum([Iron_mg[foods.index(i)] * food_vars[i] for i in foods]) <= req["Iron_mg"]['max']

prob1 += lpSum([Sodium_mg[foods.index(i)] * food_vars[i] for i in foods]) >= req["Sodium_mg"]['min']
prob1 += lpSum([Sodium_mg[foods.index(i)] * food_vars[i] for i in foods]) <= req["Sodium_mg"]['max']

prob1 += lpSum([Vit_A_IU[foods.index(i)] * food_vars[i] for i in foods]) >= req["Vit_A_IU"]['min']
prob1 += lpSum([Vit_A_IU[foods.index(i)] * food_vars[i] for i in foods]) <= req["Vit_A_IU"]['max']

prob1 += lpSum([Vit_C_IU[foods.index(i)] * food_vars[i] for i in foods]) >= req["Vit_C_IU"]['min']
prob1 += lpSum([Vit_C_IU[foods.index(i)] * food_vars[i] for i in foods]) <= req["Vit_C_IU"]['max']

prob1 += lpSum([Calories[foods.index(i)] * food_vars[i] for i in foods]) >= req["Calories"]['min']
prob1 += lpSum([Calories[foods.index(i)] * food_vars[i] for i in foods]) <= req["Calories"]['max']

prob1 += lpSum([Protein_g [foods.index(i)] * food_vars[i] for i in foods]) >= req["Protein_g"]['min']
prob1 += lpSum([Protein_g [foods.index(i)] * food_vars[i] for i in foods]) <= req["Protein_g"]['max']

prob1 += lpSum([Dietary_Fiber_g[foods.index(i)] * food_vars[i] for i in foods]) >= req["Dietary_Fiber_g"]['min']
prob1 += lpSum([Dietary_Fiber_g[foods.index(i)] * food_vars[i] for i in foods]) <= req["Dietary_Fiber_g"]['max']

prob1 += lpSum([Total_Fat_g[foods.index(i)] * food_vars[i] for i in foods]) >= req["Total_Fat_g"]['min']
prob1 += lpSum([Total_Fat_g[foods.index(i)] * food_vars[i] for i in foods]) <= req["Total_Fat_g"]['max']

prob1 += lpSum([Cholesterol_mg[foods.index(i)] * food_vars[i] for i in foods]) >= req["Cholesterol_mg"]['min']
prob1 += lpSum([Cholesterol_mg[foods.index(i)] * food_vars[i] for i in foods]) <= req["Cholesterol_mg"]['max']


food_chosen = LpVariable.dicts("Chosen",food_vars,0,1,cat='Integer') # Extra variable as binary variables to add condition for part a
for f in food_vars:
    prob1 += food_vars[f]>= food_chosen[f]*0.1 
    prob1 += food_vars[f]<= food_chosen[f]*100000

In [24]:
prob1.solve()
print("Status:", LpStatus[prob1.status])
print("\n")
print("Following food and quantities should be included in diet...")
print("\n")
for v in prob1.variables():
    if v.varValue> 0:
        print(v.name, "=", v.varValue)

Status: Optimal


Following food and quantities should be included in diet...


Chosen_Celery,_Raw = 1.0
Chosen_Frozen_Broccoli = 1.0
Chosen_Lettuce,Iceberg,Raw = 1.0
Chosen_Oranges = 1.0
Chosen_Poached_Eggs = 1.0
Chosen_Popcorn,Air_Popped = 1.0
Food_Celery,_Raw = 52.64371
Food_Frozen_Broccoli = 0.25960653
Food_Lettuce,Iceberg,Raw = 63.988506
Food_Oranges = 2.2929389
Food_Poached_Eggs = 0.14184397
Food_Popcorn,Air_Popped = 13.869322


**Conclusion** - here we put another constraint such that if a food is selected than at least 0.1 unit should be there. We can see in the solition that it satisfy this constraint.

#### Part (B) one of the two should be selected - Both Frozen Broccoli or 'Celery, Raw' should never be selected

In [25]:
# Adding binary cconntraint on variables  for brocalli and other one
prob1 += food_chosen['Frozen Broccoli']+food_chosen['Celery, Raw']<=1
prob1.solve()
print("Status:", LpStatus[prob1.status])
print("\n")
print("Following food and quantities should be included in diet...")
print("\n")
for v in prob1.variables():
    if v.varValue> 0:
        print(v.name, "=", v.varValue)

Status: Optimal


Following food and quantities should be included in diet...


Chosen_Celery,_Raw = 1.0
Chosen_Lettuce,Iceberg,Raw = 1.0
Chosen_Oranges = 1.0
Chosen_Peanut_Butter = 1.0
Chosen_Poached_Eggs = 1.0
Chosen_Popcorn,Air_Popped = 1.0
Food_Celery,_Raw = 43.154119
Food_Lettuce,Iceberg,Raw = 80.919121
Food_Oranges = 3.0765161
Food_Peanut_Butter = 2.0464575
Food_Poached_Eggs = 0.14184397
Food_Popcorn,Air_Popped = 13.181772


**Conclusion** - These are the food quantities that should be served for fulfilling daily requirements and minimizinng costs.
